In [ ]:
import os
#uncomment to us the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from calibration import CalibrationSystem, SparseModel
from calibration.categorical import AltCalibrationSystem
import gpflow
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
from calibration import synthetic
%matplotlib inline
from calibration.categorical import compute_posterior_probs, print_confusion_matrices
from calibration.baseline_categorical_methods import collaborative_filtering_SVD,evaluate,get_trust_weights,printresults

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Changing calibration...

In [ ]:
np.set_printoptions(precision=2)
N = 300
priorp = np.array([0.333,0.333,0.334])
truebees = np.random.choice(3,N,p=priorp)
times = []
#when people saw the bee
#times.append((np.linspace(0,1,N)+np.random.rand(N))/2)
#times.append((np.linspace(0,1,N)+np.random.rand(N))/2)
times.append(np.linspace(0,1,N))
times.append(np.linspace(0,1,N))
times.append(np.linspace(0,1,N))
times.append(np.linspace(0,1,N))
#times = [np.round(t) for t in times]

#times.append(np.linspace(0,1,10))
#times.append(np.linspace(0,1,10))
#times.append(np.linspace(0,1,10))

conf_matricesA = []
conf_matricesA.append(np.array([[1,0,0],[0,1,0],[0,0,1]]))
conf_matricesA.append(np.array([[1,1,1],[1,1,1],[1,1,1]]))
conf_matricesA.append(np.array([[1,1,0],[1,1,0],[0,0,1]]))
conf_matricesA.append(np.eye(3))

conf_matricesB = []
conf_matricesB.append(np.array([[1,0,0],[0,1,0],[0,0,1]]))
conf_matricesB.append(np.array([[1,0,0],[0,1,0],[0,0,1]]))
conf_matricesB.append(np.array([[1,0,1],[0,1,0],[1,0,1]]))
conf_matricesB.append(np.eye(3))

conf_matricesC = []
conf_matricesC.append(np.array([[1.1,1,1],[1,1.1,1],[1,1.1,1]]))
conf_matricesC.append(np.array([[1,0,0],[0,1,0],[0,0,1]]))
conf_matricesC.append(np.array([[1.1,1,1],[1,1.1,1],[1,1.1,1]]))
conf_matricesC.append(np.eye(3))

def genguess(conf_matrix,truebee):
    return np.random.choice(conf_matrix.shape[0],1,p=conf_matrix[truebee,:])

D = np.full([len(truebees),len(times)],np.nan)
trueconfmatrices = []
for person in range(len(times)):
    truecms = []
    for i,(tb,time) in enumerate(zip(truebees,times[person])):
        cm = conf_matricesA[person]*max(2.0*(0.5-time),0) + conf_matricesB[person]*max(1.0-2*np.abs(0.5-time),0) + conf_matricesC[person]*max(2.0*(time-0.5),0)
        cm = (cm.T/np.sum(cm,1)).T
        fill = False
        if np.random.rand()<0.6:
            fill = True
        #if person == len(times)-1:
        #    fill = True
        if person == len(times)-2:
            if np.all(np.isnan(D[i,:person])):
                fill = True
        if fill:
            D[i,person] = genguess(cm,tb)
        truecms.append(cm)
    trueconfmatrices.append(truecms)
refsensor = np.array([0,0,0,1])

X,Y = synthetic.buildXY_from_D(D)
#replaces index column with times
X = np.c_[[times[p][idx] for p,idx in zip(X[:,1],X[:,0])],[times[p][idx] for p,idx in zip(X[:,2],X[:,0])],X[:,1:]]

In [ ]:
import pickle
tempD = pickle.load(open('D.pkl','rb'))

In [ ]:
np.sum(~np.isnan(tempD),0)

In [ ]:
np.sum(np.isnan(tempD[:,-1])),np.sum(~np.isnan(tempD[:,-1]))

In [ ]:
import pickle
if False:
    pickle.dump(X,open('X.pkl','wb'))
    pickle.dump(Y,open('Y.pkl','wb'))
    pickle.dump(D,open('D.pkl','wb'))
    pickle.dump(priorp,open('priorp.pkl','wb'))
    pickle.dump(refsensor,open('refsensor.pkl','wb'))
    pickle.dump(truebees,open('truebees.pkl','wb'))
    pickle.dump(times,open('times.pkl','wb'))
    pickle.dump(trueconfmatrices,open('trueconfmatrices.pkl','wb'))
else:
    X = pickle.load(open('X.pkl','rb'))
    Y = pickle.load(open('Y.pkl','rb'))
    D = pickle.load(open('D.pkl','rb'))
    priorp = pickle.load(open('priorp.pkl','rb'))
    refsensor = pickle.load(open('refsensor.pkl','rb'))
    truebees = pickle.load(open('truebees.pkl','rb'))
    times = pickle.load(open('times.pkl','rb'))
    trueconfmatrices = pickle.load(open('trueconfmatrices.pkl','rb'))

In [ ]:
np.set_printoptions(precision=2)

Z = np.linspace(0,1,2)[:,None]
ks = [gpflow.kernels.RBF(25,0.5)] ##suggest 0.5->0.75?
kernelindices = [[0]*len(refsensor)]*(len(priorp)**2)

cs = AltCalibrationSystem(X, Y, Z, refsensor, int(len(priorp)**2), ks, kernelindices,lr=0.05,minibatchsize=30,priorp=priorp)

In [ ]:
elbo_record,samps = cs.run(its=500,samples=500)

In [ ]:
plt.plot(elbo_record)


In [ ]:
allprobs = []
for sensor,time in enumerate(times):
    probs = []
    print(sensor)
    for t in time:
        print(".",end="")
        probs.append( compute_posterior_probs(cs,sensors=[sensor],t=t,num_samps=1000)[0] )
    allprobs.append(probs)
allprobs = np.array(allprobs)

In [ ]:
import pickle
pickle.dump(allprobs,open('synthmodelresult_rbf.pkl','wb'))
#allprobs = pickle.load(open('synthmodelresult_rbf.pkl','rb'))

In [ ]:
#print_confusion_matrices(allprobs,conf_matricesA)
colfil_results = collaborative_filtering_SVD(D,len(priorp),refsensor)
trust = get_trust_weights(D,len(priorp))
results = evaluate(D,truebees,priorp,allprobs,colfil_results,cs.refsensor,trust)
pickle.dump(results,open('results_RBF.pkl','wb'))

In [ ]:
printresults(results)

In [ ]:
np.set_printoptions(precision=2)
for person in [0,1,2]:
    plt.figure(figsize=[10,10])
    ploti = 0
    #68 -> 16 - 84
    #95 -> 2.5 - 97.5
    perc = [2.5,97.5]#[16,84]
    for truebee in range(3):
        for predbee in range(3):
            ploti+=1
            plt.subplot(3,3,ploti)

            cibottom = np.percentile(allprobs[person,:,:,:,:],perc[0],1)[:,truebee,predbee]
            citop = np.percentile(allprobs[person,:,:,:,:],perc[1],1)[:,truebee,predbee]
            plt.fill_between(times[person],cibottom,citop,color='green',alpha=0.3)
            plt.plot(times[person],np.median(allprobs[person,:,:,:,:],1)[:,truebee,predbee])
            plt.plot(times[person],np.array(trueconfmatrices)[person,:,truebee,predbee])
            plt.ylim([0,1])
            plt.savefig('synth_person%d.pdf' % person)
            ##plt.plot(times[person],np.mean(allprobs[person,:,:,:,:],1)[:,truebee,predbee])
            #plt.plot(times[person],np.percentile(allprobs[person,:,:,:,:],2.5,1)[:,truebee,predbee])
            #plt.plot(times[person],np.percentile(allprobs[person,:,:,:,:],97.5,1)[:,truebee,predbee])

In [ ]:
np.set_printoptions(precision=2)

Z = np.linspace(0,1,1)[:,None]
ks = [gpflow.kernels.Bias(25)] ##suggest 0.5->0.75?
kernelindices = [[0]*len(refsensor)]*(len(priorp)**2)

cs = AltCalibrationSystem(X, Y, Z, refsensor, int(len(priorp)**2), ks, kernelindices,lr=0.05,minibatchsize=30,priorp=priorp)

In [ ]:
elbo_record,samps = cs.run(its=500,samples=500)

In [ ]:
allprobs = []
for sensor,time in enumerate(times):
    probs = []
    print(sensor)
    for t in time:
        print(".",end="")
        probs.append( compute_posterior_probs(cs,sensors=[sensor],t=t,num_samps=1000)[0] )
    allprobs.append(probs)
allprobs = np.array(allprobs)

In [ ]:
import pickle
pickle.dump(allprobs,open('synthmodelresult_bias.pkl','wb'))

In [ ]:
#print_confusion_matrices(allprobs,conf_matricesA)
colfil_results = collaborative_filtering_SVD(D,len(priorp),refsensor)
trust = get_trust_weights(D,len(priorp))
results = evaluate(D,truebees,priorp,allprobs,colfil_results,cs.refsensor,trust)
pickle.dump(results,open('results_bias.pkl','wb'))

In [ ]:
printresults(results)

In [ ]:
print_confusion_matrices(allprobs[:,0,:,:,:],conf_matricesB)

In [ ]:
ts = np.array([np.sum(truebees==i) for i in range(3)])
ts = ts/np.sum(ts)
ts*100

In [ ]:
np.sum(np.isnan(D[:,-1]))

# Real Data

In [ ]:
#import pandas as pd
#import numpy as np
#from calibration import CalibrationSystem, SparseModel, synthetic
#import gpflow
#import matplotlib.pyplot as plt
#import tensorflow as tf
import os
#uncomment to us the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from calibration import CalibrationSystem, SparseModel
from calibration.categorical import AltCalibrationSystem
import gpflow
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
from calibration import synthetic
%matplotlib inline
from calibration.categorical import compute_posterior_probs, print_confusion_matrices
from calibration.baseline_categorical_methods import collaborative_filtering_SVD,evaluate,get_trust_weights,printresults

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
%matplotlib inline

In [ ]:
rare = ['Bombus bohemicus', 'Bombus jonellus', 'Bombus sylvestris', 'Bombus muscorum','Bombus campestris', 'Apis mellifera','Bombus monticola']
    #rare.extend(['Bombus pratorum','Bombus hypnorum','Bombus hortorum']) #temp to make it quick
D, priorp, truebees = synthetic.build_D_from_csv('beelabels.csv',rare)
print("%d non-DNN labels" % np.sum(~np.isnan(D[:,:-2])))
print("%d DNN labels" % np.sum(~np.isnan(D[:,-2])))
print("%d DNN labels" % np.sum(~np.isnan(D[:,:-1])))

In [ ]:
for p in range(D.shape[1]-1):
    d = D[:,p]
    inc = (~np.isnan(d)) & (~np.isnan(D[:,-1]))
    t = np.sum(inc)
    s = np.sum(d[inc]==D[inc,-1])
    print("%d: %d/%d = %0.2f%%" % (p,s,t,100*s/t))

In [ ]:
np.set_printoptions(precision=2)

allresults = []
for crossval in range(5):
    rare = ['Bombus bohemicus', 'Bombus jonellus', 'Bombus sylvestris', 'Bombus muscorum','Bombus campestris', 'Apis mellifera','Bombus monticola']
    #rare.extend(['Bombus pratorum','Bombus hypnorum','Bombus hortorum']) #temp to make it quick
    D, priorp, truebees = synthetic.build_D_from_csv('beelabels.csv',rare)
    
    #Cross val split here...
    #this is where we want to nan some of the ref values in D.
    D[crossval::5,-1]=np.nan

    
    refsensor = np.zeros(D.shape[1])
    refsensor[-1] = 1
    X,Y = synthetic.buildXY_from_D(D)

    Z = np.linspace(0,1,1)[:,None]
    ks = [gpflow.kernels.Bias(100)]
    kernelindices = [[0]*len(refsensor)]*(len(priorp)**2)

    cs = AltCalibrationSystem(X, Y, Z, refsensor, int(len(priorp)**2), ks, kernelindices,lr=0.04,minibatchsize=10,priorp=priorp)

    #%timeit -r1 -n1 
    elbo_record,samps = cs.run(its=250,samples=500)
    #TODO: Need to rewrite this method - as modified to handle changing conf_matrices...
    allprobs = compute_posterior_probs(cs)
    #print_confusion_matrices(allprobs,conf_matrices)
    colfil_results = collaborative_filtering_SVD(D,len(priorp),refsensor)
    trust = get_trust_weights(D,len(priorp))
    #TODO: Need to rewrite this method - as modified to handle changing conf_matrices...
    results = evaluate(D,truebees,priorp,allprobs,colfil_results,cs.refsensor,trust)
    allresults.append(results)

In [ ]:
for results in allresults:
    printresults(results)

In [ ]:
import pickle
pickle.dump(allresults,open('allresults_2fold.pkl','wb'))
#allresults = pickle.load(open('allresults.pkl','rb'))

In [ ]:
combined = {}
for results in allresults:
    for a,b in results.items():
        if a not in combined: combined[a] = []
        combined[a].extend(b)

In [ ]:
from calibration.baseline_categorical_methods import printresults

printresults(combined)

The output was:
```
#5fold xval
=====PERCENT CORRECT=====
Calibration Method            84.2% correct
Collaborative Filtering       57.9% correct
Baseline (most guessed)       80.0% correct
Baseline (", trust weighted)  82.1% correct
Baseline (", prior weighted)  78.9% correct
Baseline (most common)        53.7% correct
======NLPD======
Calibration Method            59.89
Collaborative Filtering       inf
Baseline (most guessed)       69.41
Baseline (", trust weighted)  91.74
Baseline (", prior weighted)  98.42
Baseline (most common)        130.50
```

# Run on all data to get confusion matrices

In [ ]:
rare = ['Bombus bohemicus', 'Bombus jonellus', 'Bombus sylvestris', 'Bombus muscorum','Bombus campestris', 'Apis mellifera','Bombus monticola']
#rare.extend(['Bombus pratorum','Bombus hypnorum','Bombus hortorum']) #temp to make it quick
D, priorp, truebees = synthetic.build_D_from_csv('beelabels.csv',rare)

In [ ]:
print(", ".join(["\emph{%s}"% r for r in rare]))

In [ ]:
!head 'beelabels.csv'

In [ ]:
priorp

In [ ]:
np.sum(~np.isnan(D[:,-1]))

In [ ]:
np.sum(np.any(~np.isnan(D[~np.isnan(D[:,-1]),:-1]),1))

In [ ]:
np.set_printoptions(precision=2)
a = np.c_[[np.sum(D[:,-1]==i) for i in range(13)],priorp*np.sum(~np.isnan(D))]
(a/np.sum(a,0))

In [ ]:
np.sum(~np.isnan(D[:,-1]))

In [ ]:

rare = ['Bombus bohemicus', 'Bombus jonellus', 'Bombus sylvestris', 'Bombus muscorum','Bombus campestris', 'Apis mellifera','Bombus monticola']
#rare.extend(['Bombus pratorum','Bombus hypnorum','Bombus hortorum']) #temp to make it quick
D, priorp, truebees = synthetic.build_D_from_csv('beelabels.csv',rare)

refsensor = np.zeros(D.shape[1])
refsensor[-1] = 1
X,Y = synthetic.buildXY_from_D(D)

Z = np.linspace(0,1,1)[:,None]
ks = [gpflow.kernels.Bias(100)]
kernelindices = [[0]*len(refsensor)]*(len(priorp)**2)

cs = AltCalibrationSystem(X, Y, Z, refsensor, int(len(priorp)**2), ks, kernelindices,lr=0.04,minibatchsize=10,priorp=priorp)

elbo_record,samps = cs.run(its=250,samples=500)
#TODO: Need to rewrite this method - as modified to handle changing conf_matrices...
allprobs = compute_posterior_probs(cs)
print_confusion_matrices(allprobs,None)
#colfil_results = collaborative_filtering_SVD(D,len(priorp),refsensor)
#trust = get_trust_weights(D,len(priorp))
#results = evaluate(D,truebees,priorp,allprobs,colfil_results,cs.refsensor,trust)
#allresults.append(results)

```
Record from last run:

Person 0
--mean posterior--
[[70. 11.  8. 22. 11. 21.]
 [ 2. 17.  8. 10. 14.  5.]
 [ 3. 20. 55. 14. 20. 12.]
 [ 3. 11.  8.  7. 10.  8.]
 [ 5. 19. 14. 29. 27. 17.]
 [18. 21.  7. 17. 17. 36.]]
95% CIs:
[[5. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]] 

 [[ 99. 100.  49. 100.  96. 100.]
 [ 14.  99.  73.  98.  95.  60.]
 [ 30. 100.  98. 100.  99.  99.]
 [ 27.  99.  70.  84.  95.  83.]
 [ 43. 100.  79. 100. 100.  99.]
 [ 92. 100.  46. 100. 100. 100.]]
-------------------
Person 1
--mean posterior--
[[97.  6.  2.  3. 15. 87.]
 [ 1. 64.  1.  3. 11.  2.]
 [ 1.  9. 94.  4. 25.  4.]
 [ 1.  9.  1. 86.  6.  3.]
 [ 1.  7.  1.  3. 37.  4.]
 [ 0.  6.  1.  1.  6.  1.]]
95% CIs:
[[88.  0.  0.  0.  0. 40.]
 [ 0.  9.  0.  0.  0.  0.]
 [ 0.  0. 83.  0.  0.  0.]
 [ 0.  0.  0. 51.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]] 

 [[ 99.  37.  10.  16.  96. 100.]
 [  6.  99.   6.  32.  96.  21.]
 [  3.  67.  99.  38.  96.  31.]
 [  3.  48.   7. 100.  47.  21.]
 [  4.  70.  13.  26.  99.  44.]
 [  2.  49.   5.   9.  57.   7.]]
-------------------
Person 2
--mean posterior--
[[95. 23. 20. 13. 27. 34.]
 [ 1. 18.  5.  7. 11. 12.]
 [ 1. 19. 51. 13. 28. 15.]
 [ 1. 14.  9. 53. 13. 22.]
 [ 1. 13.  9.  8. 15. 10.]
 [ 2. 13.  6.  5.  6.  9.]]
95% CIs:
[[72.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]] 

 [[100. 100.  95.  76. 100. 100.]
 [  4. 100.  31.  69.  97.  99.]
 [  6. 100. 100.  83. 100. 100.]
 [  6.  99.  75.  99.  99. 100.]
 [  6.  99.  57.  71.  99.  99.]
 [ 15. 100.  66.  58.  91. 100.]]
-------------------
Person 3
--mean posterior--
[[80. 18.  3.  5. 21. 30.]
 [ 1.  6.  3.  3. 13.  4.]
 [ 3. 18. 86.  4. 25.  4.]
 [ 2. 23.  3. 77. 18.  3.]
 [ 2. 12.  2.  5. 11.  5.]
 [13. 22.  2.  6. 13. 54.]]
95% CIs:
[[36.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0. 51.  0.  0.  0.]
 [ 0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]] 

 [[ 98. 100.  17.  34. 100.  85.]
 [ 11.  93.  34.  37.  98.  38.]
 [ 15. 100.  98.  57. 100.  52.]
 [  8. 100.  33. 100. 100.  18.]
 [ 13.  99.  14.  47.  98.  48.]
 [ 57. 100.  29.  52. 100.  98.]]
-------------------
Person 4
--mean posterior--
[[88. 40.  4.  5. 16. 40.]
 [ 1.  5.  1.  4.  6.  4.]
 [ 4. 36. 87.  7. 31. 11.]
 [ 4.  6.  6. 74. 28.  8.]
 [ 1.  6.  1.  5.  9.  8.]
 [ 2.  7.  2.  5. 10. 29.]]
95% CIs:
[[52.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  1. 33.  0.  0.  0.]
 [ 0.  0.  0. 11.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]] 

 [[ 98.  94.  19.  29. 100.  97.]
 [  9.  43.   5.  59.  98.  44.]
 [ 33.  95. 100.  44. 100.  76.]
 [ 22.  43.  49.  99. 100.  73.]
 [  7.  61.   4.  59.  80.  80.]
 [ 15.  66.  25.  32.  98.  99.]]
-------------------
Person 5
--mean posterior--
[[76. 71.  2.  4. 16. 55.]
 [ 3.  9.  1.  8. 12.  1.]
 [ 1.  4. 78.  8. 14.  2.]
 [ 1.  4.  1. 60. 12.  1.]
 [18.  7. 17. 17. 41. 13.]
 [ 1.  4.  1.  3.  5. 28.]]
95% CIs:
[[62.  1.  0.  0.  0. 16.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  0. 42.  0.  0.  0.]
 [ 0.  0.  0. 23.  0.  0.]
 [ 9.  0.  2.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  3.]] 

 [[ 87.  99.   8.  27.  99.  88.]
 [  8.  98.   8.  43.  92.  10.]
 [  5.  36.  96.  35.  81.   9.]
 [  2.  32.   7.  95.  76.   6.]
 [ 34.  70.  56.  60. 100.  67.]
 [  4.  28.   7.  20.  53.  71.]]
-------------------

```